In [5]:
import pyspark

In [6]:
from pyspark.sql.functions import split,col

In [7]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/16 22:51:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/16 22:51:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/16 22:51:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/16 22:51:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/02/16 22:51:27 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [8]:
df = spark.read.csv("../Datasets/WineDataset.csv", inferSchema="True", header="True")

In [9]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Capacity: string (nullable = true)
 |-- Grape: string (nullable = true)
 |-- Secondary Grape Varieties: string (nullable = true)
 |-- Closure: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Characteristics: string (nullable = true)
 |-- Per bottle / case / each: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- ABV: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- Vintage: string (nullable = true)
 |-- Appellation: string (nullable = true)



In [29]:
# Fetch price and store in a new Column 

df = df.withColumn("bottle_price", split(split(df.Price, " ")[0], "£")[1].cast("Float"))


In [30]:
df.columns

['Title',
 'Description',
 'Price',
 'Capacity',
 'Grape',
 'Secondary Grape Varieties',
 'Closure',
 'Country',
 'Unit',
 'Characteristics',
 'Per bottle / case / each',
 'Type',
 'ABV',
 'Region',
 'Style',
 'Vintage',
 'Appellation',
 'bottle_price']

In [37]:
# Count of Bottle titles with Null price

df.where(df.bottle_price.isNull()).select(col("Title"),col("bottle_price")).distinct().count()

142

In [56]:
from pyspark.sql.functions import sum, avg, count

In [110]:
# Bottle with max price, average price, sum price

df.groupBy("Country",df.Grape).avg("bottle_price").select(df.Grape,df.Country.alias("C"),col("avg(bottle_price)").alias("p")).sort(df.Grape).show()

+--------------------+--------------------+------------------+
|               Grape|                   C|                 p|
+--------------------+--------------------+------------------+
|                NULL|             Germany|              20.0|
|                NULL|                NULL| 73.81333287556966|
|                NULL|               Italy| 8.993333180745443|
|                NULL|               Spain|13.989999771118164|
|                NULL|                 9.8|              NULL|
|                NULL|              Mexico|              33.0|
|                NULL|        South Africa|16.989999771118164|
|                NULL|            Scotland|              80.0|
|                NULL|      United Kingdom| 5.989999771118164|
| The Wine Advocate."|  Cabernet Sauvignon|              NULL|
|           a mineral|                75CL|              NULL|
| black pepper and...|                75CL|              NULL|
| blackcurrant and...|          Pinot Noir|            

In [97]:
df.select(col("Grape").isNull()).count()

1410

In [103]:
df.select(df.Grape).count()

1410

In [111]:
df.sort("Grape",descending=False).select("Grape").show()

+-----+
|Grape|
+-----+
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
+-----+
only showing top 20 rows



In [113]:
df.select("Grape").distinct().show()

+--------------------+
|               Grape|
+--------------------+
|              Shiraz|
|Lambrusco Graspar...|
|                75CL|
|      Sauvignon Gris|
|       Feteasca Alba|
|            Pinotage|
|         Agiorgitiko|
| plum and summer ...|
|            Dolcetto|
|            Loureiro|
| citrusy characte...|
|               Greco|
|            Viognier|
|        Nero D'Avola|
|          Chardonnay|
|           Carménère|
|        Black Muscat|
|            Sémillon|
| cedar and chocol...|
|           Castelão |
+--------------------+
only showing top 20 rows



In [115]:
df.select("Country","Title","Grape").orderBy("Grape").show()

+-------+--------------------+-----+
|Country|               Title|Grape|
+-------+--------------------+-----+
|   NULL|Abbotts & Delauna...| NULL|
|   NULL|Louis Jadot 'Les ...| NULL|
|   NULL|Oyster Bay Sauvig...| NULL|
|   NULL|Definition Chabli...| NULL|
|   NULL|The Macallan Sher...| NULL|
|   NULL|Definition Zinfan...| NULL|
|   NULL|Bouvet Ladubay Sa...| NULL|
|   NULL|Klein Constantia ...| NULL|
|   NULL|The Ned 'Waihopai...| NULL|
|   NULL|Peter Lehmann 'Th...| NULL|
|   NULL|Villa Maria 'Priv...| NULL|
|   NULL|Vilarnau ‘Gaudi’ ...| NULL|
|   NULL|The Ned ‘Skyscrap...| NULL|
|   NULL|Pardon et Fils Fl...| NULL|
|   NULL|La Gioiosa Prosec...| NULL|
|   NULL|Rustenberg Cabern...| NULL|
|   NULL|LB7 Red 2020/21, ...| NULL|
|   NULL|Thierry Delaunay ...| NULL|
|   NULL|Nyetimber 'Classi...| NULL|
|   NULL|Rosé Classics 6 B...| NULL|
+-------+--------------------+-----+
only showing top 20 rows



In [116]:
df.count()

1410

In [122]:
df.select("Country").distinct().show()

+--------------------+
|             Country|
+--------------------+
|                75CL|
|Petit Verdot, Merlot|
|              1.5LTR|
| and can even be ...|
|            Viognier|
|             Germany|
|          Chardonnay|
|              France|
|              Greece|
|           Argentina|
|               Chile|
|               Italy|
|          Pinot Noir|
|        Natural Cork|
|               Spain|
|                 9.8|
|                 9.4|
|                 USA|
|              Mexico|
|             Georgia|
+--------------------+
only showing top 20 rows



In [124]:
df=df.dropna()

In [125]:
df

DataFrame[Title: string, Description: string, Price: string, Capacity: string, Grape: string, Secondary Grape Varieties: string, Closure: string, Country: string, Unit: string, Characteristics: string, Per bottle / case / each: string, Type: string, ABV: string, Region: string, Style: string, Vintage: string, Appellation: string, bottle_price: float]

In [126]:
df.count()

208

In [127]:
df.show()

+--------------------+--------------------+-----------------+--------+------------------+-------------------------+--------------+---------+----+--------------------+------------------------+-----+----------+--------------------+--------------------+-------+--------------------+------------+
|               Title|         Description|            Price|Capacity|             Grape|Secondary Grape Varieties|       Closure|  Country|Unit|     Characteristics|Per bottle / case / each| Type|       ABV|              Region|               Style|Vintage|         Appellation|bottle_price|
+--------------------+--------------------+-----------------+--------+------------------+-------------------------+--------------+---------+----+--------------------+------------------------+-----+----------+--------------------+--------------------+-------+--------------------+------------+
|Miraval Rosé 2021...|Miraval shot to f...|£22.99 per bottle|    75CL|          Cinsault|     Syrah, Grenache, ...|  Natu

In [128]:
df.write.csv("../Datasets/Wine_clean.csv")

In [10]:
df = df + df

TypeError: unsupported operand type(s) for +: 'DataFrame' and 'DataFrame'